<div class = "alert alert-danger">
    
**Un notebook se lit en exécutant toutes les cellules de code au fur et à mesure.**  
    
Pour cela il y a le bouton <kbd>&#9658; Exécuter</kbd> dans la barre d'outils ou le raccourci clavier <kbd>Shift</kbd>+<kbd>Enter</kbd>
    
</div>

In [ ]:
import matplotlib.pyplot as plt
import math
import cmath

# Réponses du filtre numérique

- [1. Réponse indicielle](#indic)
- [2. Réponse fréquentielle](#freq)

Le filtre RC d'équation différentielle $0.001s'(t)+s(t)=e(t)$ peut être approximé numériquement par l'algorithme (pour $T_e=0.1 ms$):

$$s_n = 0.091e_n + 0.909s_{n-1}$$

## 1. Réponse indicielle<a name="indic"></a>

Comparons les signaux de sortie des systèmes analogique et numérique lorsque le signal d'entrée est un échelon d'amplitude 10 V

In [ ]:
# Constantes
Te = 0.0001 # 0.1 ms
nb = 100 # ne calculer que 100 échantillons

# Variables
s = [0]*nb  # nb échantillons de sortie (à calculer)
e = [10]*nb # nb échantillons d'entrée initialisés à 10
t = [Te*i for i in range(nb)] # valeurs des instants d'échantillonnage

<div class="alert alert-warning">
    <b>Note:</b> comme son programme s'exécute indéfiniment (boucle sans fin), un microcontrôleur ne mémorise pas tous les échantillons de sortie et d'entrée. Il ne conserve que:
    
- l'échantillon de sortie courant (par exemple, pour envoyer au C.N.A)
    
- les échantillons d'entrée et de sortie précédents utilisés dans l'algorithme (ici 1 échantillon d'entrée et 1 échantillon de sortie).
</div>

- le système analogique a pour solution mathématique le signal $s(t)=10(1-e^{-1000t})$

In [ ]:
s_analogique = [10*(1-math.exp(-1000*ti)) for ti in t]

- la réponse indicielle du système numérique se calcule à l'aide de l'algorithme:

In [ ]:
# Algorithme
s[0] = 0.091 * e[0]
for n in range(1,100): # Remplacé par while(True) pour un uC
    s[n] = 0.091*e[n] + 0.909*s[n-1]

In [ ]:
# Afficher les 10 premiers échantillons de sortie
s[0:10]

Dessinons les 2 réponses dans un même graphe:

In [ ]:
fig, ax = plt.subplots()

# Réponse indicielle numérique
ax.scatter(t,s,marker='.',label='algorithme')

# Réponse indicielle analogique
ax.plot(t,s_analogique, 'r', label='réponse analogique')

# Configuration du graphe
ax.set_xlabel('Temps [s]')
ax.set_ylabel('Tension [V]')
ax.legend()
ax.grid()
plt.show()

On constate que, mis à part les premiers instants, la réponse indicielle du système numérique suit bien celle du système analogique.

## 2. Réponse fréquentielle<a name="freq"></a>

Calculons la transmittance en z de l'algorithme $s_n = 0.091e_n + 0.909s_{n-1}$:

<small>
$$\begin{align}
S(z) = 0.091E(z) + 0.909z^{-1}S(z)
\Longleftrightarrow \left(1-0.909z^{-1}\right)S(z) = 0.091E(z)
\end{align}$$
</small>

et donc $H(z)=\dfrac{S(z)}{E(z)}=\dfrac{0.091}{1-0.909z^{-1}}$

<small>Expression en puissances de z positives:
$H(z)=\dfrac{0.091z}{z-0.909}$

Rem: 1 pôle $|0.909|<1$ &rarr; le filtre est stable</small>

<div class="alert alert-info">
On admet que la transmittance isochrone équivalente du système numérique s'obtient en effectuant le changement de variable:

$$\underline{H}(jf) = H_{num}\left(z=e^{2\pi j\frac{f}{f_e}}\right)$$

- Le module $A(f)=|\underline{H}(jf)|$ correspond à l'amplification

- $G(f)=20\log(A(f))$ correspond au gain.
</div>

Pour l'algorithme d'approximation du circuit RC, on a:

$$\underline{H}(jf) = \dfrac{0.091}{1-0.909e^{-2\pi j\frac{f}{f_e}}}$$

<div class="alert alert-warning">
Il ne s'agit pas d'une fonction de transfert analogique habituelle, car elle ne s'exprime pas en fonction des puissances de f (ou de $\omega$).
</div>

Calculons ces grandeurs:

In [ ]:
# Constante
fe = 1/Te # 10 kHz

In [ ]:
# Axe des fréquences et transmittance
f = [j*x for j in [1,10,100,1000] for x in range(1,10) ]
h = [0.091/(1-0.909*cmath.exp(-2j*cmath.pi*fi/fe)) for fi in f]

Dessinons le diagramme de Bode en amplification:

In [ ]:
# Amplification
a = [abs(hi) for hi in h]

fig, ax = plt.subplots()
ax.plot(f, a)
ax.set_xlim(0,5000) # Règle de Shannon (fe=10 kHz)
ax.grid()
ax.set_xlabel('Fréquence [Hz]')
ax.set_ylabel('Amplification')
plt.show()

Dessinons le diagramme de Bode en gain:

In [ ]:
g = [20*cmath.log10(ai) for ai in a]

fig, ax = plt.subplots()
ax.plot(f, g)
ax.set_xlim(0,5000) # Règle de Shannon (fe=10 kHz)
ax.grid()
ax.set_xlabel('Fréquence [Hz]')
ax.set_ylabel('Gain [dB]')
plt.show()

**Conclusion :** Il s'agit d'un filtre passe-bas. Si on zoome, on constate qu'il a une fréquence de coupure à -3dB de 150 Hz.

(alors que le filtre analogique a une fréquence de coupure $f_c=\frac{1}{2\pi RC}=$159 Hz)